# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [157]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [158]:
# 2) seu código aqui
sinasc.isna().sum()

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [159]:
# 3) seu código aqui
colunas_interessantes = sinasc.loc[:, ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]

colunas_interessantes.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [160]:
# 4) seu código aqui
print(sinasc.shape)

sinasc = sinasc.dropna(subset = ['APGAR5'])

print(sinasc.shape)

(27028, 69)
(26925, 69)


In [161]:
# 5) seu código aqui
print(sinasc[sinasc[['ESTCIVMAE', 'CONSULTAS']] == 9].sum().sum())

sinasc['ESTCIVMAE'] = sinasc['ESTCIVMAE'].fillna(9)
sinasc['CONSULTAS'] = sinasc['CONSULTAS'].fillna(9)

print(sinasc[sinasc[['ESTCIVMAE', 'CONSULTAS']] == 9].sum().sum())

1746.0
4581.0


In [162]:
# 6) Seu código aqui
print(sinasc['QTDFILVIVO'].isna().sum())

sinasc['QTDFILVIVO'] = sinasc['QTDFILVIVO'].fillna(0)

print(sinasc['QTDFILVIVO'].isna().sum())

1566
0


In [163]:
# 7) seu código aqui
print(sinasc.isna().sum().sum())


for i in sinasc.columns:
    if sinasc[i].dtype != 'O':
        mediana = sinasc[i].median()
        
        sinasc[i] = sinasc[i].fillna(mediana)
    else:
        sinasc[i] = sinasc[i].fillna('não preenchido')
        
print(sinasc.isna().sum().sum())

118897
26925


In [164]:
# 8) seu código aqui
sinasc.loc[(sinasc['APGAR5'] >= 0)  & (sinasc['APGAR5'] <= 3), 'passAsfixia'] = 'normal'
sinasc.loc[(sinasc['APGAR5'] >= 4)  & (sinasc['APGAR5'] <= 5), 'passAsfixia'] = 'asfixia leve'
sinasc.loc[(sinasc['APGAR5'] >= 6)  & (sinasc['APGAR5'] <= 7), 'passAsfixia'] = 'asfixia moderada'
sinasc.loc[(sinasc['APGAR5'] >= 8)  & (sinasc['APGAR5'] <= 10), 'passAsfixia'] = 'asfixia severa'

print(sinasc[sinasc.APGAR5 == 6].loc[:289,'passAsfixia'])
print(sinasc[sinasc.APGAR5 == 3].loc[:4384,'passAsfixia'])
print(sinasc[sinasc.APGAR5 == 5].loc[:1118,'passAsfixia'])

205    asfixia moderada
289    asfixia moderada
Name: passAsfixia, dtype: object
185     normal
4384    normal
Name: passAsfixia, dtype: object
418     asfixia leve
1118    asfixia leve
Name: passAsfixia, dtype: object


In [165]:
# 9) seu código aqui

# Obtendo apenas as colunas que estão em CamelCase
lista_colunas = sinasc.loc[:, 'munResStatus':'passAsfixia'].columns 

# Obtendo as colunas que estão com a string em maiúsculo e colocando-as em minúsculo
lista_colunas_maiusculas = list(sinasc.loc[:,'ORIGEM':'CONTADOR'].columns)
lista_colunas_snake = list(i.lower() for i in lista_colunas_maiusculas)


# Transformando as colunas que estão em CamelCase para snake_case
for coluna in lista_colunas:
    coluna[0].lower()
    for i in coluna[1:]: 
        if i.isupper():
            coluna = coluna.replace(f'{i}', f'_{i.lower()}')
            
    lista_colunas_snake.append(coluna)
    

    
sinasc.columns = lista_colunas_snake

sinasc.head(10)

,origem,codestab,codmunnasc,locnasc,idademae,estcivmae,escmae,codocupmae,qtdfilvivo,qtdfilmort,...,contador,mun_res_status,mun_res_tipo,mun_res_nome,mun_res_uf,mun_res_lat,mun_res_lon,mun_res_alt,mun_res_area,pass_asfixia
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,999992.0,0.0,0.0,...,1,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025,asfixia severa
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,2,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025,asfixia severa
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,3,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025,asfixia severa
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,4,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,-12.13178,-61.85308,397.0,3958.273,asfixia severa
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025,asfixia severa
5,1,2516500.0,110001,1,33,5.0,1 a 3 anos,621005.0,2.0,1.0,...,6,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025,asfixia severa
6,1,2516500.0,110001,1,37,1.0,8 a 11 anos,999992.0,1.0,0.0,...,7,ATIVO,MUNIC,Novo Horizonte do Oeste,Rondônia,-11.70974,-61.99439,230.0,843.446,asfixia severa
7,1,2516500.0,110001,1,27,2.0,12 anos ou mais,514120.0,0.0,0.0,...,8,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025,asfixia severa
8,1,2679477.0,110001,1,20,5.0,8 a 11 anos,621005.0,0.0,1.0,...,9,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025,asfixia severa
9,1,2679477.0,110001,1,30,2.0,1 a 3 anos,999992.0,1.0,1.0,...,10,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025,asfixia severa
